In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import requests
from datetime import datetime, timedelta
import pandas as pd

from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
API_KEY = "ayNClqmZuOOBeL6lFjA348UfsA0jpzazy8pjyXsQ"
API_IDS = {'wind': 'EBA.MISO-ALL.NG.WND.H',
        'solar': 'EBA.MISO-ALL.NG.SUN.H',
        'hydro': 'EBA.MISO-ALL.NG.WAT.H',
        'coal': 'EBA.MISO-ALL.NG.COL.H',
        'natural_gas': 'EBA.MISO-ALL.NG.NG.H',
        'nuclear': 'EBA.MISO-ALL.NG.NUC.H',
        'other': 'EBA.MISO-ALL.NG.OTH.H'}
HEADERS = {'Content-Type': 'application/json'}
USERNAME = input()
PASSWORD = input()

In [ ]:
def get_historical_pi_data(name, start_time, end_time, interval):
    base_url = 'https://itsnt2259.iowa.uiowa.edu/piwebapi/search/query?q=name:'
    url = base_url + name
    query = requests.get(url, auth=(USERNAME, PASSWORD), headers=HEADERS).json()
    self_url = query['Items'][0]['Links']['Self']
    point = requests.get(self_url, auth=(USERNAME, PASSWORD), headers=HEADERS).json()
    data_url = point['Links']['InterpolatedData']
    data_url = data_url + '/?startTime=-' + start_time + '&endTime=-' + end_time + '&interval=' + interval
    data = requests.get(data_url, auth=(USERNAME, PASSWORD), headers=HEADERS).json()
    return data

def get_historical_df(names, startTime, endTime, interval):
    df = pd.DataFrame()
    for i, name in enumerate(names):
        data = get_historical_pi_data(name, startTime, endTime, interval)
        items = data['Items']
        if i == 0:
            times = [int(datetime.strptime(item['Timestamp'][:-2], '%Y-%m-%dT%H:%M:%S.%f').timestamp()//3600) for item in items]
            df['Time'] = times
        validateData = lambda x: min(x, 50) if (isinstance(x, (int, float)) and x >= 0) else 0
        vals = [validateData(item['Value']) for item in items]
        df[name] = pd.DataFrame(vals)
    return df

In [ ]:
def getTotalLoad(startTime="520w", endTime="1d", interval="1h"):
    names = ['PP_Electric_Purch', 'PP_Electric_Gen']
    df = get_historical_df(names, startTime, endTime, interval)
    df['Load'] = df['PP_Electric_Purch'] + df['PP_Electric_Gen']
    return df
data = getTotalLoad()
print(data)

In [ ]:
def convert2matrix(data_arr, look_back):
    X, Y = [], []
    for i in range(len(data_arr)-look_back):
        d=i+look_back  
        X.append(data_arr[i:d,0])
        Y.append(data_arr[d,0])
    return np.array(X), np.array(Y)

load_data = data[['Load']]
load_data.head()


In [ ]:
# Split data set into testing dataset and train dataset
train_size, val_size = int(load_data.shape[0]*0.95), int(load_data.shape[0]*0.01)
train, val, test = \
    load_data.values[0:train_size, :], load_data.values[train_size:train_size+val_size, :], load_data.values[train_size+val_size:, :]

# setup look_back window 
look_back = 24

#convert dataset into right shape in order to input into the DNN
trainX, trainY = convert2matrix(train, look_back)
valX, valY = convert2matrix(val, look_back)
testX, testY = convert2matrix(test, look_back)

print("Training data:", train.shape)
print("Validation data:", val.shape)
print("Testing data:", test.shape)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
def model_dnn(look_back):
    model=Sequential()
    model.add(Dense(units=32, input_dim=look_back, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',  optimizer='adam',metrics = ['mse', 'mae'])
    return model

def model_loss(history):
    fig, ax = plt.subplots(1,2, figsize=(15,5))
    ax[0].plot(history.history['loss'], label='Train Loss')
    ax[0].set_title('Training Loss')
    ax[0].set_ylabel('loss')
    ax[0].set_xlabel('epochs')
    ax[0].legend(loc='upper right')

    ax[1].plot(history.history['val_loss'], label='Val Loss')
    ax[1].set_title('Validation Loss')
    ax[1].set_ylabel('loss')
    ax[1].set_xlabel('epochs')
    ax[1].legend(loc='upper right')
    plt.show()


In [ ]:
model   = model_dnn(look_back)
history = model.fit(trainX, trainY, epochs=100, batch_size=30, verbose=1, validation_data=(valX, valY), \
                callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)], shuffle=False)

In [ ]:
train_score = model.evaluate(trainX, trainY, verbose=0)
print('Train Root Mean Squared Error(RMSE): %.2f; Train Mean Absolute Error(MAE) : %.2f ' % (np.sqrt(train_score[1]), train_score[2]))
test_score = model.evaluate(testX, testY, verbose=0)
print('Test Root Mean Squared Error(RMSE): %.2f; Test Mean Absolute Error(MAE) : %.2f ' % (np.sqrt(test_score[1]), test_score[2]))
model_loss(history)

In [ ]:
test_predict = model.predict(testX)

In [ ]:
def prediction_plot(testY, test_predict):
    len_prediction=[x for x in range(len(testY))]
    plt.figure(figsize=(20,5))
    plt.plot(len_prediction, testY, marker='.', label="actual")
    plt.plot(len_prediction, test_predict, 'r', label="prediction")
    plt.tight_layout()
    plt.subplots_adjust(left=0.07)
    plt.ylabel('Load', size=15)
    plt.xlabel('Time step', size=15)
    plt.legend(fontsize=15)
    plt.show()

prediction_plot(testY, test_predict)

In [ ]:
model.save("load_predictor")

In [ ]:
reconstructed_model = keras.models.load_model("load_predictor")

In [ ]:
np.testing.assert_allclose(
    model.predict(testX), reconstructed_model.predict(testX)
)